# Projekat 2
- Rok za predaju je termin odbrane u toku 2. kolokvijumske nedelje
- Projekat je moguće raditi u paru ili samostalno
- Svaka stavka nosi po 10 bodova

Koristeći PyCuda okurženje napisati CUDA program za (matrično) množenje matrica.

1. Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)
2. Probram koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.
3. Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije)
4. Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 57.6MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 471kB 62.2MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4535180 sha256=c1d2e7a96f26d6b9c6c246b4a55953154fa317665261a11135f390758b2b9e87
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.1-py2.py3-none-any.whl size=59602 sha256=c46fe4335cf027257521444a25d39cb8c1343888b66852c3bd5a28b5abb09c7d
  Stored in directory: /root/.cache/pip/wheels/6f/da/1b/946775a88291378182ed92c9800d6d0ebc2a554cb89829cc24
  Created wheel for mako: filename=Mako-1.1.1-cp36-none-any.whl size=75410 sha256=e159a6dc0ca64072c597095dd72449162aba2c7cad573e6c11d3b28ce29b6a91
  Stored in directory: /root/.cache/pip/wheels/ff/50/a9/0ddeed8679a1fb65bf4677cb9c92701828b2c1821e22ef72fd
Successfully built pycuda pytools mako


In [0]:
!nvidia-smi

Thu Dec 19 16:52:04 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

1. Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixOneBlock(float *a,float *b, float *c, int colsA,int colsB){
              int idxC = threadIdx.x+threadIdx.y*colsB;
              int idxA = threadIdx.y*colsA;
              int idxB = threadIdx.x;

              int i=0;

              for(i;i<colsA;i++){
                c[idxC] += a[idxA+i]*b[idxB];
                idxB += colsB;
              }
            }
        """)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(10): warning: expression has no effect




In [3]:
import numpy as np

a = np.random.randn(20,10).astype(dtype=np.float32)
b = np.random.randn(10,10).astype(dtype=np.float32)
# print(b)
res = a.dot(b)

c = np.zeros_like(res)
# print(c)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixOneBlock")
func(a_gpu,b_gpu,c_gpu,np.int32(a.shape[1]),np.int32(b.shape[1]),block=(np.int(b.shape[1]),np.int(a.shape[0]),1),grid=(1,1,1))

cuda.memcpy_dtoh(c,c_gpu)

print(res)
print("\n")
print(c)

print("Dobijen ocekivan rezultat: ",(c==res).all())


[[-6.61319261e-03 -1.03212273e+00 -6.60954905e+00 -2.94812679e+00
   3.08738732e+00 -6.59219146e-01 -1.69506133e+00  1.19594723e-01
  -2.52241397e+00 -2.90766716e+00]
 [ 3.11278701e+00 -5.93282747e+00 -3.01727200e+00  8.34733307e-01
  -8.91571331e+00  1.76132631e+00  5.49504900e+00 -1.51824605e+00
   2.08662438e+00  2.05112982e+00]
 [ 3.40328884e+00  1.59361362e+00 -6.54365396e+00 -6.70715761e+00
  -6.87209272e+00  2.76411700e+00  6.42351675e+00  2.31044388e+00
  -3.65087605e+00 -5.73620200e-01]
 [ 1.22270966e+00  3.05270743e+00 -7.36995935e-01 -4.12804365e+00
  -4.18360758e+00  2.33640695e+00  3.93337345e+00  7.53668189e-01
  -8.81967366e-01 -3.06086540e-01]
 [-2.44525552e-01  1.42606705e-01  5.84352136e-01  2.60837126e+00
   5.04781246e-01  1.44806087e+00  1.81941319e+00  4.85533619e+00
   2.20984578e+00 -1.15936124e+00]
 [-5.93612480e+00 -5.62515450e+00 -3.00647712e+00  2.00549340e+00
  -5.43911457e-01 -9.67341602e-01 -8.01352143e-01 -1.62194312e-01
   2.78397727e+00  8.48718047e-01

2. Program koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixMultipleBlocksaaa(float *a,float *b, float *c,int colsB,int rowsA,int colsA){
              long idxC = threadIdx.x + blockDim.x*blockIdx.x + threadIdx.y*colsB + blockDim.y*blockIdx.y*colsB;
              long idxA = (threadIdx.y + blockDim.y*blockIdx.y)*colsA;
              long idxB = blockDim.x*blockIdx.x+threadIdx.x;

              if(idxB > colsB) return;
              if(threadIdx.y + blockDim.y*blockIdx.y > rowsA) return;
             

              for(int i = 0;i<colsA;i++){
                c[idxC] += a[idxA+i]*b[idxB];
                idxB += colsB;
              }
            }
        """)

In [6]:
import numpy as np
import math

a = np.random.randn(90,64).astype(dtype=np.float32)
b = np.random.randn(64,50).astype(dtype=np.float32)

res = a.dot(b)

c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixMultipleBlocksaaa")
func(a_gpu, b_gpu, c_gpu,np.int32(b.shape[1]),np.int32(a.shape[0]),np.int32(a.shape[1]),block=(32, 32, 1), grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c,c_gpu)
print(res)
print("\n")
print(c)
print("Dobijen ocekivan rezultat: ",(c==res).all())

[[  8.059529    -2.9514744   19.793642   ...   0.77876675  -1.9025819
    4.662512  ]
 [  5.1823907   -8.657698     5.429327   ...  -3.2701514    2.2115963
  -20.331495  ]
 [  7.914715    13.15319     -6.9358788  ... -14.764278     5.5255246
   -8.185326  ]
 ...
 [ -0.3132923   18.790424     5.3004994  ...   1.3879613   -4.4121203
    5.031723  ]
 [ -4.4500666    7.9562087  -11.3538885  ...  11.753639   -12.153389
    1.573964  ]
 [-12.711066     2.6455061    6.8246527  ...  12.291165   -13.488839
    1.153004  ]]


[[  8.059529    -2.9514744   19.793642   ...   0.77876675  -1.9025819
    4.662512  ]
 [  5.1823907   -8.657698     5.429327   ...  -3.2701514    2.2115963
  -20.331495  ]
 [  7.914715    13.15319     -6.9358788  ... -14.764278     5.5255246
   -8.185326  ]
 ...
 [ -0.3132923   18.790424     5.3004994  ...   1.3879613   -4.4121203
    5.031723  ]
 [ -4.4500666    7.9562087  -11.3538885  ...  11.753639   -12.153389
    1.573964  ]
 [-12.711066     2.6455061    6.8246527  ...

3. Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije)    

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixMultipleBlocksSharedMemory(float *a,float *b, float *c,int rowsA, int colsA,
                                                                  int rowsB,int colsB,int rowsC,int colsC){
              float cValue = 0;

              int row = blockIdx.y * blockDim.y + threadIdx.y;
              int col = blockIdx.x * blockDim.y + threadIdx.x;

              __shared__ float aVal[32][32];
              __shared__ float bVal[32][32];

              for(int i = 0; i < (blockDim.y + colsA - 1)/blockDim.y;i++){
                if(i*blockDim.y + threadIdx.x < colsA && row < rowsA){
                  aVal[threadIdx.y][threadIdx.x] = a[row*colsA + i*blockDim.y + threadIdx.x];
                } else {
                  aVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                if(i * blockDim.y + threadIdx.y < rowsB && col < colsB){
                  bVal[threadIdx.y][threadIdx.x] = b[(i*blockDim.y + threadIdx.y)* colsB + col];
                } else {
                  bVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                __syncthreads();

                for(int j = 0 ; j < blockDim.y ; ++j){
                  cValue += aVal[threadIdx.y][j] * bVal[j][threadIdx.x];
                }

                __syncthreads();

                if(row < rowsC && col < colsC){
                  c[((blockIdx.y * blockDim.y) + threadIdx.y) * colsC + (blockIdx.x * blockDim.x) + threadIdx.x] = cValue;
                }                
              } 
            }
        """)

In [8]:
import numpy as np
import math

a = np.random.randn(40,64).astype(dtype=np.float32)
b = np.random.randn(64,90).astype(dtype=np.float32)

res = a.dot(b)

c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixMultipleBlocksSharedMemory")
func(a_gpu, b_gpu, c_gpu,np.int32(a.shape[0]),np.int32(a.shape[1]),np.int32(b.shape[0]),np.int32(b.shape[1]),
     np.int32(a.shape[0]),np.int32(b.shape[1]),block=(32, 32, 1), grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c,c_gpu)
print(res)
print("\n")
print(c)
print("Dobijen ocekivan rezultat: ",(c==res).all())

[[  5.765109    12.437745    12.543252   ...  -0.48804048 -18.68554
   -2.8234823 ]
 [ -1.1579032   16.959457    -0.67889977 ... -14.982883    10.15992
   -6.843334  ]
 [-11.283044     6.9160376   10.162625   ...  12.919693    -0.49158838
    4.5014462 ]
 ...
 [-14.676378    12.977117    13.151093   ...  16.911741     3.1119206
   -3.4816144 ]
 [ -7.7833877    3.412491    -7.264991   ...  -7.1473784   -2.6236696
   -6.525996  ]
 [  6.107656     1.0148576   -1.7636495  ...   2.045672    -9.665419
    2.4342937 ]]


[[  5.765109    12.437745    12.543252   ...  -0.48804048 -18.68554
   -2.8234823 ]
 [ -1.1579032   16.959457    -0.67889977 ... -14.982883    10.15992
   -6.843334  ]
 [-11.283044     6.9160376   10.162625   ...  12.919693    -0.49158838
    4.5014462 ]
 ...
 [-14.676378    12.977117    13.151093   ...  16.911741     3.1119206
   -3.4816144 ]
 [ -7.7833877    3.412491    -7.264991   ...  -7.1473784   -2.6236696
   -6.525996  ]
 [  6.107656     1.0148576   -1.7636495  ...   2

4. Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 

        __global__ void multiplyMatrixMultipleBlocksSharedMemory(float *a,float *b, float *c,int rowsA, int colsA,
                                                                  int rowsB,int colsB,int rowsC,int colsC){
              float cValue = 0;

              int row = blockIdx.y * blockDim.y + threadIdx.y;
              int col = blockIdx.x * blockDim.y + threadIdx.x;

              __shared__ float aVal[32][32];
              __shared__ float bVal[32][32];

              for(int i = 0; i < (colsA - 1)/blockDim.x + 1;i++){
                if(i*blockDim.y + threadIdx.x < colsA && row < rowsA){
                  aVal[threadIdx.y][threadIdx.x] = a[row*colsA + i*blockDim.x + threadIdx.x];
                } else {
                  aVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                if(i * blockDim.x + threadIdx.y < colsB && col < rowsB){
                  bVal[threadIdx.y][threadIdx.x] = b[(i*blockDim.y + threadIdx.y)+ colsB * col];
                } else {
                  bVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                __syncthreads();

                for(int j = 0 ; j < blockDim.y ; ++j){
                  cValue += aVal[threadIdx.y][j] * bVal[j][threadIdx.x];
                }

                __syncthreads();

                if(row < rowsC && col < colsC){
                  c[row*colsC+col] = cValue;
                }                 
              } 
            }
        """)

In [10]:
import numpy as np
import math

a = np.random.randn(40,64).astype(dtype=np.float32)
b = np.random.randn(90,64).astype(dtype=np.float32)

a_rows = np.int32(a.shape[0])
a_cols = np.int32(a.shape[1])

b_rows = np.int32(b.shape[0])
b_cols = np.int32(b.shape[1])

res = a.dot(np.transpose(b))

c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixMultipleBlocksSharedMemory")
func(a_gpu, b_gpu, c_gpu,a_rows,a_cols,b_rows,b_cols,
     a_rows,b_rows,block=(32, 32, 1), grid=(math.ceil(b_rows/32), math.ceil(a_rows/32), 1))

cuda.memcpy_dtoh(c,c_gpu)
print(res)
print("\n")
print(c)
print("Dobijen ocekivan rezultat: ",(c==res).all())

[[  4.433159   -12.212397     9.166139   ...  -6.384378    14.973863
   -6.887016  ]
 [ -1.743611    -3.532164    14.190903   ...   7.9171133   -6.845802
    7.449575  ]
 [  2.9409366   -2.977142    -4.4708385  ...  -6.2097006   -3.0482185
   15.085486  ]
 ...
 [  8.189446    -1.9610255    1.7757114  ... -13.651409    -0.24192235
    4.1073117 ]
 [ -1.8791156    7.0345764    0.6736483  ... -11.351215    -5.9611197
    4.4481745 ]
 [  4.0102777   -9.1871       7.4591684  ...  -4.826018     2.7697608
    4.7740536 ]]


[[  4.433159   -12.212397     9.166139   ...  -6.384378    14.973863
   -6.887016  ]
 [ -1.743611    -3.532164    14.190903   ...   7.9171133   -6.845802
    7.449575  ]
 [  2.9409366   -2.977142    -4.4708385  ...  -6.2097006   -3.0482185
   15.085486  ]
 ...
 [  8.189446    -1.9610255    1.7757114  ... -13.651409    -0.24192235
    4.1073117 ]
 [ -1.8791156    7.0345764    0.6736483  ... -11.351215    -5.9611197
    4.4481745 ]
 [  4.0102777   -9.1871       7.4591684  ..